In [1]:
%%bash
fastqc -help


            FastQC - A high throughput sequence QC analysis tool

SYNOPSIS

	fastqc seqfile1 seqfile2 .. seqfileN

    fastqc [-o output dir] [--(no)extract] [-f fastq|bam|sam] 
           [-c contaminant file] seqfile1 .. seqfileN

DESCRIPTION

    FastQC reads a set of sequence files and produces from each one a quality
    control report consisting of a number of different modules, each one of 
    which will help to identify a different potential type of problem in your
    data.
    
    If no files to process are specified on the command line then the program
    will start as an interactive graphical application.  If files are provided
    on the command line then the program will run with no user interaction
    required.  In this mode it is suitable for inclusion into a standardised
    analysis pipeline.
    
    The options for the program as as follows:
    
    -h --help       Print this help file and exit
    
    -v --version    Print the version of the program and exit

In [2]:
pwd

'/home/babin/bgi_test_exome'

In [4]:
ls -lh ./1_input_data/

total 8.4G
-rw-r--r-- 1 babin babin 4.2G Jun 29 13:25 raw_62_1.fq.gz
-rw-r--r-- 1 babin babin 4.2G Jun 29 13:42 raw_62_2.fq.gz


### 1. Running FastQC

In [6]:
%%bash
fastqc \
./1_input_data/raw_62_1.fq.gz \
./1_input_data/raw_62_2.fq.gz \
-q \
-t 2 \
-o ./2_fastqc_out/


#### FastQC plots explanation:


- https://dnacore.missouri.edu/PDF/FastQC_Manual.pdf
- https://rtsf.natsci.msu.edu/genomics/tech-notes/fastqc-tutorial-and-faq/

### 2. Trimming reads
- we have poort "Per base sequence content"
- so we cut out 10 bases from the head

#### trimmomatic manual
- http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/TrimmomaticManual_V0.32.pdf
- http://www.usadellab.org/cms/?page=trimmomatic

In [4]:
%%bash
trimmomatic \
PE \
-threads 32 \
./1_input_data/raw_62_1.fq.gz \
./1_input_data/raw_62_2.fq.gz \
./3_trim_out/forw_paired_62_1.fq.gz \
./3_trim_out/forw_unpaired_62_1.fq.gz \
./3_trim_out/rev_paired_62_2.fq.gz \
./3_trim_out/rev_unpaired_62_2.fq.gz \
HEADCROP:10 \
MINLEN:36



TrimmomaticPE: Started with arguments:
 -threads 32 ./1_input_data/raw_62_1.fq.gz ./1_input_data/raw_62_2.fq.gz ./3_trim_out/forw_paired_62_1.fq.gz ./3_trim_out/forw_unpaired_62_1.fq.gz ./3_trim_out/rev_paired_62_2.fq.gz ./3_trim_out/rev_unpaired_62_2.fq.gz HEADCROP:10 MINLEN:36
Quality encoding detected as phred33
Input Read Pairs: 48115979 Both Surviving: 48115979 (100.00%) Forward Only Surviving: 0 (0.00%) Reverse Only Surviving: 0 (0.00%) Dropped: 0 (0.00%)
TrimmomaticPE: Completed successfully


### 3. mapping reads on reference by bwa

#### 3.1 indexing reference sequence

In [6]:
%%bash
bwa index 0_ref/GRCh37_latest_genomic.fna.gz

[bwa_index] Pack FASTA... 61.34 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=6469669378, availableWord=467229400
[BWTIncConstructFromPacked] 10 iterations done. 99999986 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 199999986 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 299999986 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 399999986 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 499999986 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 599999986 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 699999986 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 799999986 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 899999986 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 999999986 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 

In [7]:
ls 3_trim_out/

forw_paired_62_1.fq.gz    rev_paired_62_2.fq.gz
forw_unpaired_62_1.fq.gz  rev_unpaired_62_2.fq.gz


#### 3.2 mapping reads

In [9]:
%%bash
bwa mem \
-t 16 \
0_ref/GRCh37_latest_genomic.fna.gz \
3_trim_out/forw_paired_62_1.fq.gz \
3_trim_out/rev_paired_62_2.fq.gz > \
4_bwa_out/bgi_exome.sam 

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 1684212 sequences (160000140 bp)...
[M::process] read 1684212 sequences (160000140 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (6, 674003, 26, 8)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (193, 226, 267)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (45, 415)
[M::mem_pestat] mean and std.dev: (231.94, 55.51)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 489)
[M::mem_pestat] analyzing insert size distribution for orientation RF...
[M::mem_pestat] (25, 50, 75) percentile: (103, 206, 504)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 1306)
[M::mem_pestat] mean and std.dev: (267.09, 267.33)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 1707)
[M::mem_pestat] skip orientation RR a

### 4. dealing with bam file

- manual : http://www.htslib.org/workflow/ (see "WGS/WES Mapping to Variant Calls - Version 1.0")

#### 4.1 fixmate : clean up read pairing information and flags after bwa mapping
- `-O` output format
- `--threads` number of threads to use


In [12]:
%%bash
samtools fixmate -O bam \
--threads 16 \
4_bwa_out/bgi_exome.sam \
4_bwa_out/bgi_exome_fixmate_62.bam

#### 4.2 sort  reads by genome position
- `-O` output format
- `--threads` number of threads to use
- `-m` memory for each thread in Gb
- `-o` output file

In [15]:
%%bash
samtools sort \
-O bam \
--threads 8 \
-m 1G \
-o ./4_bwa_out/bgi_exome_fixmate_sorted_62.bam \
./4_bwa_out/bgi_exome_fixmate_62.bam 


[bam_sort_core] merging from 24 files and 8 in-memory blocks...


#### 4.3 indexing sorted bam
- `-@` number of threads to use

In [19]:
%%bash
samtools index \
./4_bwa_out/bgi_exome_fixmate_sorted_62.bam \
-@ 16

### 5. bcftools variant calling

- `mpileup` generates genotype likelihoods
- `call`  makes actuall calling
- `-f`  reference file
- `-m`  default calling method
- `-v`  output only variant sites
- `-O`  selecting output format 'b' compressed BCF; 'u' uncompressed BCF; 'z' compressed VCF; 'v' uncompressed VCF

In [26]:
%%bash
bcftools mpileup \
-f 0_ref/GRCh37_latest_genomic.fna \
./4_bwa_out/bgi_exome_fixmate_sorted_62.bam \
--threads 16 \
| \
bcftools call \
-mv -Ov \
-o ./5_variants/bgi_test.vcf \
--threads 16

Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 250


### 7. annotating our vcf with rs IDs from dbsnp vcf 

#### 7.1 compressing vcf file prior to annotation 

In [8]:
%%bash
bgzip -c \
./6_variants_annot_with_dbsnp/bgi_test.vcf > \
./6_variants_annot_with_dbsnp/bgi_test.vcf.gz

#### 7.2 indexing compressed vcf

In [9]:
%%bash
tabix -p \
vcf \
./6_variants_annot_with_dbsnp/bgi_test.vcf.gz

#### 7.2 indexing dbsnp vcf 

In [13]:
%%bash 
tabix -p \
vcf \
./0_annot_dbsnp_37/GRCh37_latest_dbSNP_all.vcf.gz

#### 7.3 annotating our vcf with rs IDs from dbsnp
- `-a` file with annotations
- `-c` columns to take from file with annotations
- `-o` output file with annotations

In [14]:
%%bash
bcftools annotate \
-a ./0_annot_dbsnp_37/GRCh37_latest_dbSNP_all.vcf.gz \
-c ID \
-o ./6_variants_annot_with_dbsnp/bgi_test_annotated.vcf.gz \
./6_variants_annot_with_dbsnp/bgi_test.vcf.gz